In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-08-07 15:52:13.952347


In [18]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
dutils = discogsUtils()
col    = collections(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Warning! Local Discog Directory /Users/tgadfort/Documents/music/Discog is not Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/special exists
/Volumes/Music/Discog/artist-special exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value


# Collections

In [ ]:
%load_ext autoreload
%autoreload
col = collections(disc)
#col.parseCollections(force=False)
#col.createCollectionDBs()
col.parseCollectionsForAlbums(force=True)
#from webUtils import getHTML
#bsdata = getHTML('/Volumes/Music/Discog/collections/Rock & Roll-US-2010-1.p')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found 0 known album collections.
Found 0 previous merged files.
Found 62564 downloaded collection files for py3=True
Found 62564 downloaded collection files not processed.
Parsing 0/62564: /Volumes/Music/Discog/collections/Abstract-Argentina-1960-1.p
Parsing 100/62564: /Volumes/Music/Discog/collections/Abstract-Netherlands-2000-1.p
Parsing 200/62564: /Volumes/Music/Discog/collections/Acid House-Denmark-2000-1.p
Parsing 300/62564: /Volumes/Music/Discog/collections/Acid Jazz-Belgium-2000-1.p
Parsing 400/62564: /Volumes/Music/Discog/collections/Acid Rock-Canada-2010-1.p
Parsing 500/62564: /Volumes/Music/Discog/collections/Acid-Australia-2010-1.p
Parsing 600/62564: /Volumes/Music/Discog/collections/Acid-Unknown-2010-1.p
Parsing 700/62564: /Volumes/Music/Discog/collections/Acoustic-Mexico-1970-1.p
Parsing 800/62564: /Volumes/Music/Discog/collections/African-Austria-2010-1.p
Parsing 900/62564: /Volumes/Mu

  --> This file is 2.8MB.
Saved data to /Volumes/Music/Discog/collections-db/albums-collections-5.p
  --> This file is 2.8MB.
Parsing 6100/62564: /Volumes/Music/Discog/collections/Canada-2017-Electronic-2.p
Parsing 6200/62564: /Volumes/Music/Discog/collections/Canada-2019-Latin-3.p
Parsing 6300/62564: /Volumes/Music/Discog/collections/Celtic-Denmark-1990-1.p
Parsing 6400/62564: /Volumes/Music/Discog/collections/Celtic-Unknown-1990-1.p
Parsing 6500/62564: /Volumes/Music/Discog/collections/Chanson-Argentina-1970-1.p
Parsing 6600/62564: /Volumes/Music/Discog/collections/Chanson-Japan-1990-1.p
Parsing 6700/62564: /Volumes/Music/Discog/collections/Chanson-USSR-2010-1.p
Parsing 6800/62564: /Volumes/Music/Discog/collections/Chicago Blues-UK & Europe-2010-1.p
Parsing 6900/62564: /Volumes/Music/Discog/collections/Chiptune-US-2000-1.p
Parsing 7000/62564: /Volumes/Music/Discog/collections/Choral-Portugal-1970-1.p
Saving 7001 collections to /Volumes/Music/Discog/collections-db/collectionsKnownAlbu

  --> This file is 5.1MB.
Saved data to /Volumes/Music/Discog/collections-db/albums-collections-11.p
  --> This file is 5.1MB.
Parsing 12100/62564: /Volumes/Music/Discog/collections/Education-UK-2010-1.p
Parsing 12200/62564: /Volumes/Music/Discog/collections/Electric Blues-Japan-1960-1.p
Parsing 12300/62564: /Volumes/Music/Discog/collections/Electro House-Belgium-2000-1.p
Parsing 12400/62564: /Volumes/Music/Discog/collections/Electro-Brazil-2000-1.p
Parsing 12500/62564: /Volumes/Music/Discog/collections/Electro-UK & Europe-2000-1.p
Parsing 12600/62564: /Volumes/Music/Discog/collections/Emo-Netherlands-2000-1.p
Parsing 12700/62564: /Volumes/Music/Discog/collections/Ethereal-Italy-1990-1.p
Parsing 12800/62564: /Volumes/Music/Discog/collections/Euro House-Brazil-1990-1.p
Parsing 12900/62564: /Volumes/Music/Discog/collections/Euro-Disco-Argentina-2000-1.p
Parsing 13000/62564: /Volumes/Music/Discog/collections/Eurodance-Japan-2000-1.p
Saving 13001 collections to /Volumes/Music/Discog/collec

  --> This file is 3.9MB.
Saved data to /Volumes/Music/Discog/collections-db/albums-collections-17.p
  --> This file is 3.9MB.
Parsing 18100/62564: /Volumes/Music/Discog/collections/Hard Techno-Unknown-1990-1.p
Parsing 18200/62564: /Volumes/Music/Discog/collections/Hardcore Hip-Hop-Australia-1980-1.p
Parsing 18300/62564: /Volumes/Music/Discog/collections/Hardcore Hip-Hop-Unknown-1980-1.p
Parsing 18400/62564: /Volumes/Music/Discog/collections/Hardstyle-Argentina-2010-1.p
Parsing 18500/62564: /Volumes/Music/Discog/collections/Harmonica Blues-Germany-2010-1.p
Parsing 18600/62564: /Volumes/Music/Discog/collections/Harsh Noise Wall-US-2010-1.p
Parsing 18700/62564: /Volumes/Music/Discog/collections/Heavy Metal-Finland-1970-1.p
Parsing 18800/62564: /Volumes/Music/Discog/collections/Heavy Metal-USSR-1970-1.p
Parsing 18900/62564: /Volumes/Music/Discog/collections/Hi NRG-Switzerland-2010-1.p
Parsing 19000/62564: /Volumes/Music/Discog/collections/Highlife-Portugal-2010-1.p
Saving 19001 collection

  --> This file is 1.9MB.
Saved data to /Volumes/Music/Discog/collections-db/albums-collections-23.p
  --> This file is 1.9MB.
Parsing 24100/62564: /Volumes/Music/Discog/collections/Mariachi-Unknown-1960-1.p
Parsing 24200/62564: /Volumes/Music/Discog/collections/Medieval-Australia-1960-1.p
Parsing 24300/62564: /Volumes/Music/Discog/collections/Medieval-Norway-2010-1.p
Parsing 24400/62564: /Volumes/Music/Discog/collections/Melodic Hardcore-Austria-2000-1.p
Parsing 24500/62564: /Volumes/Music/Discog/collections/Memphis Blues-Greece-1990-1.p
Parsing 24600/62564: /Volumes/Music/Discog/collections/Merengue-Argentina-1960-1.p
Parsing 24700/62564: /Volumes/Music/Discog/collections/Metalcore-Japan-2010-1.p
Parsing 24800/62564: /Volumes/Music/Discog/collections/Military-Greece-1960-1.p
Parsing 24900/62564: /Volumes/Music/Discog/collections/Minimal Techno-Belgium-2000-1.p
Parsing 25000/62564: /Volumes/Music/Discog/collections/Minimal-Brazil-2000-1.p
Saving 25001 collections to /Volumes/Music/Dis

  --> This file is 1.5MB.
Saved data to /Volumes/Music/Discog/collections-db/albums-collections-29.p
  --> This file is 1.5MB.
Parsing 30100/62564: /Volumes/Music/Discog/collections/Pipe & Drum-Germany-1970-1.p
Parsing 30200/62564: /Volumes/Music/Discog/collections/Pipe & Drum-US-1950-1.p
Parsing 30300/62564: /Volumes/Music/Discog/collections/Poetry-Italy-1990-1.p
Parsing 30400/62564: /Volumes/Music/Discog/collections/Poetry-Unknown-2010-1.p
Parsing 30500/62564: /Volumes/Music/Discog/collections/Political-Jamaica-2000-1.p
Parsing 30600/62564: /Volumes/Music/Discog/collections/Polka-France-1990-1.p
Parsing 30700/62564: /Volumes/Music/Discog/collections/Pop Punk-Brazil-2000-1.p
Parsing 30800/62564: /Volumes/Music/Discog/collections/Pop Punk-UK & Europe-2000-1.p
Parsing 30900/62564: /Volumes/Music/Discog/collections/Pop Rap-Portugal-2000-1.p
Parsing 31000/62564: /Volumes/Music/Discog/collections/Pop Rock-Finland-2010-1.p
Saving 31001 collections to /Volumes/Music/Discog/collections-db/col

  --> This file is 208.6kB.
Saved data to /Volumes/Music/Discog/collections-db/collectionsKnownAlbums.p
  --> This file is 208.6kB.
Saving 1000 entries to /Volumes/Music/Discog/collections-db/albums-collections-35.p
Saving data to /Volumes/Music/Discog/collections-db/albums-collections-35.p
  --> This file is 4.4MB.
Saved data to /Volumes/Music/Discog/collections-db/albums-collections-35.p
  --> This file is 4.4MB.
Parsing 36100/62564: /Volumes/Music/Discog/collections/Rock & Roll-USSR-2010-1.p
Parsing 36200/62564: /Volumes/Music/Discog/collections/Rockabilly-Canada-2010-1.p
Parsing 36300/62564: /Volumes/Music/Discog/collections/Rockabilly-Russia-1960-1.p
Parsing 36400/62564: /Volumes/Music/Discog/collections/Rocksteady-Denmark-1980-1.p
Parsing 36500/62564: /Volumes/Music/Discog/collections/Rocksteady-Switzerland-1960-1.p
Parsing 36600/62564: /Volumes/Music/Discog/collections/Romani-Italy-2010-1.p
Parsing 36700/62564: /Volumes/Music/Discog/collections/Romantic-Austria-2000-1.p
Parsing 

  --> This file is 242.8kB.
Saved data to /Volumes/Music/Discog/collections-db/collectionsKnownAlbums.p
  --> This file is 242.8kB.
Saving 1000 entries to /Volumes/Music/Discog/collections-db/albums-collections-41.p
Saving data to /Volumes/Music/Discog/collections-db/albums-collections-41.p
  --> This file is 2.3MB.
Saved data to /Volumes/Music/Discog/collections-db/albums-collections-41.p
  --> This file is 2.3MB.
Parsing 42100/62564: /Volumes/Music/Discog/collections/Steel Band-Austria-1950-1.p
Parsing 42200/62564: /Volumes/Music/Discog/collections/Steel Band-US-1950-1.p
Parsing 42300/62564: /Volumes/Music/Discog/collections/Stoner Rock-Poland-1990-1.p
Parsing 42400/62564: /Volumes/Music/Discog/collections/Story-Denmark-2000-1.p
Parsing 42500/62564: /Volumes/Music/Discog/collections/Story-Spain-1950-1.p
Parsing 42600/62564: /Volumes/Music/Discog/collections/Surf-Denmark-1970-1.p
Parsing 42700/62564: /Volumes/Music/Discog/collections/Surf-Russia-1990-1.p
Parsing 42800/62564: /Volumes/

  --> This file is 278.2kB.
Saved data to /Volumes/Music/Discog/collections-db/collectionsKnownAlbums.p
  --> This file is 278.2kB.
Saving 1000 entries to /Volumes/Music/Discog/collections-db/albums-collections-47.p
Saving data to /Volumes/Music/Discog/collections-db/albums-collections-47.p
  --> This file is 9.6MB.
Saved data to /Volumes/Music/Discog/collections-db/albums-collections-47.p
  --> This file is 9.6MB.
Parsing 48100/62564: /Volumes/Music/Discog/collections/US-1960-Classical-6.p
Parsing 48200/62564: /Volumes/Music/Discog/collections/US-1960-Country-39.p
Parsing 48300/62564: /Volumes/Music/Discog/collections/US-1960-Electronic-2.p
Parsing 48400/62564: /Volumes/Music/Discog/collections/US-1960-Funk-14.p
Parsing 48500/62564: /Volumes/Music/Discog/collections/US-1960-Gospel-7.p
Parsing 48600/62564: /Volumes/Music/Discog/collections/US-1960-Jazz-Funk-4.p
Parsing 48700/62564: /Volumes/Music/Discog/collections/US-1960-Ottoman Classical-1.p
Parsing 48800/62564: /Volumes/Music/Disco

  --> This file is 11.6MB.
Saved data to /Volumes/Music/Discog/collections-db/albums-collections-53.p
  --> This file is 11.6MB.
Parsing 54100/62564: /Volumes/Music/Discog/collections/US-1990-Punk-17.p
Parsing 54200/62564: /Volumes/Music/Discog/collections/US-1990-Rhythm & Blues-9.p
Parsing 54300/62564: /Volumes/Music/Discog/collections/US-1990-Salsa-1.p
Parsing 54400/62564: /Volumes/Music/Discog/collections/US-1990-Soul-23.p


In [46]:
col.createCollectionDBsForAlbums()

Current Time is Wed Aug 07, 2019 22:13:57 for Creating Initial Album Database
/Volumes/Music/Discog/collections-db/albums-collections-0.p 	/Volumes/Music/Discog/collections-db/albums-collections-1.p 	/Volumes/Music/Discog/collections-db/albums-collections-2.p 	/Volumes/Music/Discog/collections-db/albums-collections-3.p 	Saving 109069 entries to /Volumes/Music/Discog/db/CollectionAlbumRefCounts.p
Saving data to /Volumes/Music/Discog/db/CollectionAlbumRefCounts.p
  --> This file is 2.5MB.
Saved data to /Volumes/Music/Discog/db/CollectionAlbumRefCounts.p
  --> This file is 2.5MB.
Saving 109069 entries to /Volumes/Music/Discog/db/CollectionAlbumRefArtists.p
Saving data to /Volumes/Music/Discog/db/CollectionAlbumRefArtists.p
  --> This file is 4.0MB.
Saved data to /Volumes/Music/Discog/db/CollectionAlbumRefArtists.p
  --> This file is 4.0MB.
Current Time is Wed Aug 07, 2019 22:14:05 for Done with Creating Initial Album Database
Process [{0}] took 8 seconds.


In [50]:
from collections import Counter
albcnts = Counter(getFile('/Volumes/Music/Discog/db/CollectionAlbumRefCounts.p'))
albarts = getFile('/Volumes/Music/Discog/db/CollectionAlbumRefArtists.p')

In [53]:
for item in albcnts.most_common(10):
    ref, cnt = item[0], item[1]
    aa = albarts[ref]
    print(item,aa)

('/Goat-Requiem/release/9160868', 8) ['/artist/3129107-Goat-22']
('/Spam-Allstars-Trans-Oceanic/release/9536487', 7) ['/artist/223142-Spam-Allstars']
('/PsyAviah-Introspection-Extrospection/master/383684', 6) ['/artist/194339-PsyAviah']
('/PsyAviah-Introspection-Extrospection/release/3218516', 6) ['/artist/194339-PsyAviah']
('/Saskia-Laroo-Its-Like-Jazz/release/9587624', 6) ['/artist/52393-Saskia-Laroo']
('/Virtual-Alien-LoudMouth/release/7756856', 6) ['/artist/2142329-Virtual-Alien']
('/%CE%9D%CE%AF%CE%BA%CE%BF%CF%82-%CE%9A%CE%B1%CF%81%CE%B2%CE%AD%CE%BB%CE%B1%CF%82-%CE%9C%CE%B1%CE%B8%CE%AE%CE%BC%CE%B1%CF%84%CE%B1-%CE%A6%CE%B9%CE%BB%CE%BF%CF%83%CE%BF%CF%86%CE%AF%CE%B1%CF%82/release/7879764', 6) ['/artist/959301-Νίκος-Καρβέλας', '/artist/959301-Î\x9dÎ¯ÎºÎ¿Ï\x82-Î\x9aÎ±Ï\x81Î²Î\xadÎ»Î±Ï\x82']
('/Goat-Commune/release/6117708', 6) ['/artist/3129107-Goat-22']
('/Goat-Commune/release/6116963', 6) ['/artist/3129107-Goat-22']
('/Goat-Commune/release/6113587', 6) ['/artist/3129107-Goat-22']


# New Artists

In [ ]:
%load_ext autoreload
%autoreload

from collection import Counter

refCounts = Counter(disc.getCollectionRefCountsData())
artistIDs = disc.getArtistIDToRefData()

In [2]:
errs = getFile("colErrs.json")
len(errs)

500

In [ ]:
n = len(refCounts)
nNext = len(errs)

nFalse = 0
nNone  = 0
nTrue  = 0

for i,(ref,cnts) in enumerate(refCounts.most_common()):
    if i % 2500 == 0:
        print("Processed {0}/{1}  (Errs={2})".format(i,n, len(errs)))
        
    if errs.get(ref) is not None:
        continue
        
    artistID = dutils.getArtistID(ref)
    if artistID is None:
        continue
    if artistIDs.get(artistID) is None:
        url      = arts.getArtistURL(ref)
        savename = arts.getArtistSavename(artistID)
        if isFile(savename):
            continue
            
        try:
            retval = arts.downloadArtistURL(url=url, savename=savename, parse=True, debug=True)
        except:
            retval = None
            
        if retval is None:
            nNone += 1
            errs[ref] = "None"
        if retval is False:
            nFalse += 1
            errs[ref] = "False"

    if (nNone % 100 == 0 or nFalse % 100 == 0) and (nNone > 0 and nFalse > 0):
        print("There have been {0} errors so far...".format(nNone))
        print("There have been {0} no saves so far...".format(nFalse))
        print("There have been {0} full saves so far...".format(nTrue))        
        saveFile(ifile="colErrs.json", idata=errs)


            
    if len(errs) == nNext + 250:        
        saveFile(ifile="colErrs.json", idata=errs)
        break

In [ ]:
len(errs)

In [ ]:
errs2 = getFile("colErrs.json")
len(errs2)